In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

import stripe
stripe.api_key = os.getenv('STRIPE_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile
from ChooseService import analyze_situation_sequential


times_embedding_run = 0

Running Chroma using direct local API.
loaded in 78 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
Running Chroma using direct local API.
loaded in 22 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


#### App Development Notes:

What seems to be creating the greatest latency issues?
1. The memory, which was supposed to be a token saving/context management system but may be overkill in early stage parts of the conversation
2. The classifers that are needed to preare the prompt (the prompt itself seems to run pretty fast)

#### Policies VectorStore

In [2]:
# #load in the policies to the vectorstore
# loaders = []
# docs = []

# #create a list with the names of the files to be loaded in the data/policies folder
# for file in os.listdir('data/policies'):
#     if file.endswith('.txt'):
#         loader = TextLoader('data/policies/' + file)
#         loaders.append(loader)

# for loader in loaders:
#     docs.extend(loader.load())

# if times_embedding_run < 5:
#     text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
#     documents = text_splitter.split_documents(docs)

#     embeddings = OpenAIEmbeddings(openai_api_key=key)
#     policies = Chroma.from_documents(documents, embeddings, persist_directory='data/policies-db/')
#     times_embedding_run += 1

# "Embeddings generated into policies variable"

In [2]:
embeddings = OpenAIEmbeddings(openai_api_key=key)
policies = Chroma(persist_directory='data/policies-db/', embedding_function=embeddings)

Running Chroma using direct local API.
loaded in 119 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [3]:
location_of_issue = Property('location', None, "The location of the issue", required=True, examples=['kitchen', 'bathroom', 'bedroom', 'proactive'])

special_instructions = Property('special_instructions', None, "Any special instructions for the technician", required=False, examples=['please call before arriving', 'please leave a note on the door'])

square_footage = Property('square_footage', None, "The square footage of the property", required=True, examples=['1000', '2000', '3000'])

pest_id = Property('pest_id', None, "The id number of the pest", required=True, examples=['1', '2', '3'])

issues = NaturalLanguagePool([location_of_issue, special_instructions, pest_id, square_footage], "The issues the customer is having", create_new_properties=True)

In [29]:
chat = ChatOpenAI(openai_api_key=key)
gpt4 = ChatOpenAI(openai_api_key=key, model='gpt-4')

initial_prompt = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases."""

history = DynamicChatHistory(chat, system_message=initial_prompt, additional_context="", initial_history=[SystemMessage(content="Hi! My name is George from Better Termite & Pest Control. How can I help you? :)")])

stage_one = ConversationScriptStage('Take lead of conversation', "Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.", lambda: any(isinstance(message, AIMessage) for message in history.history))

stage_two = ConversationScriptStage('Get contact information', "Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.", lambda: customer_profile.is_complete(), pool=customer_profile, show_missing=True, show_complete=True)

stage_three = ConversationScriptStage('Get customer issue', "Gather details about the customer's issue. This goal is considered complete, once you have the pest ID, location of the issue and the square footage of the property. Make sure to check if the customer is seeing multiple issues. Ask only 1 question per response.", lambda: issues.is_complete(), pool=issues, show_missing=True, show_complete=True)

sales_script = ConversationScript([stage_one, stage_two, stage_three])

agent = Conversation(history, gpt4, sales_script, customer_profile, issues, policies, log_prompt=True)

In [30]:
agent.respond('Hi name is John Smith and I am having an issue with rodents')

NEXT GOAL:  Current goal: Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
 Current goal: Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.
*Human*:  Hi name is John Smith and I am having an issue with rodents
------------------
Agent:  Hey John, bummer about th

In [31]:
agent.respond('Sure that is fine')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊
User: Sure that is fine
 
*Human*:  Sure that is fine
------------------
Agent:  Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊


In [32]:
agent.respond('Yeah I am in Alexandria')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊
User: Sure that is fine
Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊
User: Yeah I am in Alexandria
 
*Human*:  Yeah I am in Alexandria
------------------
Agent:  Hey John, thanks for sharing! Alexandria is a love

In [33]:
agent.respond('Yes it is a single family home')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊
User: Sure that is fine
Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊
User: Yeah I am in Alexandria
Agent: Hey John, thanks for sharing! Alexandria is a lovely place - minus the rodents, of course. 😄 Now, could yo

In [34]:
agent.respond('I am seeing them in my basement and my kids room')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊
User: Sure that is fine
Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊
User: Yeah I am in Alexandria
Agent: Hey John, thanks for sharing! Alexandria is a lovely place - minus the rodents, of course. 😄 Now, could yo

In [37]:
print(agent.customer_profile.natural_language())

first_name: John
last_name: Smith
phone_number: 571-431-9531
email: georgeschulz33@gmail.com
street_address: 4355 Penwood Drive
city: Alexandria
state: VA
zip: 22314



In [35]:
agent.respond('No I have no idea where they are coming in from but they are everywhere. pooping all over the house')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am having an issue with rodents
Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊
User: Sure that is fine
Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊
User: Yeah I am in Alexandria
Agent: Hey John, thanks for sharing! Alexandria is a lovely place - minus the rodents, of course. 😄 Now, could yo

In [8]:
agent.respond('My phone is 571-431-9531 and email is georgeschulz33@gmail.com')

NEXT GOAL:  Current goal: Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.
What we are missing: street_address, city, state, zip
What we have so far:
first_name: George
last_name: Schulz
phone_number: 571-431-9531
email: georgeschulz33@gmail.com
street_address: ?
city: ?
state: ?
zip: ?

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, nice to meet you! I'd love to help you out with any pest

In [9]:
agent.respond('My address is 4355 Penwood Drive Alexandria, VA 22314')

NEXT GOAL:  Current goal: Gather details about the customer's issue. This goal is considered complete, once you have the pest ID, location of the issue and the square footage of the property. Make sure to check if the customer is seeing multiple issues. Ask only 1 question per response.
What we are missing: square_footage
What we have so far:
location: Bathroom
special_instructions: Earwigs are in the bathroom
pest_id: 0
square_footage: ?

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, nice to meet you! I'd love to he

In [10]:
agent.respond('I am seeing spiders all around the house. I have seen them in the master bedroom and on the front porch. My house is 3400 square feet')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, nice to meet you! I'd love to help you out with any pest control issues you might be facing. Is it cool if I ask you a few questions to get a better understanding of your situation?
User: Sure
Agent: Awesome, thanks John! To start, what's your phone number and email address? This will help me get in touch with you if we need to chat more about your pest control needs.
User: My phone is 571-431-9531 and email is georgeschulz33@gmail.com
Agent: Thanks, Geor

In [13]:
agent.respond('Just ants right now')

NEXT GOAL:  Current goal: Gather details about the customer's issue. This goal is considered complete, once you have the pest ID, location of the issue and the square footage of the property. Make sure to check if the customer is seeing multiple issues. Ask only 1 question per response.
What we are missing: square_footage
What we have so far:
location: Kitchen
special_instructions: Ants are in the kitchen
pest_id: 1
square_footage: ?

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, great to hear you're interested in pe

In [14]:
agent.respond('Yeah its 1400 square feet. Right now I have been seeing them on my counter tops near my bannanas')

NEXT GOAL:  None
*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, great to hear you're interested in pest control! Do you mind if I ask you a few questions to better understand your situation and needs?
User: Sure
Agent: Great! Can I grab your email, phone number, street address, city, state and zip code so I can have a better understanding of your pest control needs and the services we can provide for you?
User: Yeah my email is georgeschulz33@gmail.com, phone is 571-431-9531 and address is 4355 penwood drive alexandri

In [2]:
conversation = """User: Hi name is John Smith and I am having an issue with rodents

Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊

User: Sure that is fine

Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊

User: Yeah I am in Alexandria
Agent: Hey John, thanks for sharing! Alexandria is a lovely place - minus the rodents, of course. 😄 Now, could you let me know what kind of property you're dealing with, like a house or an apartment? That'll help me suggest the right approach.

User: Yes it is a single family home

Agent: Gotcha, John! Single-family homes can sometimes be a magnet for rodents. Don't worry, we've got your back. 😎 Have you noticed any specific areas inside or outside your home where these pesky rodents seem to be hanging out the most?

User: I am seeing them in my basement and my kids room

Agent: Oh no, John! That's definitely not cool, especially in your kid's room. 😩 Let me help you out. Do you have an idea of how many rodents you've seen or any specific entry points they're using? It helps us come up with a solid plan to tackle this problem. 👊

User: No I have no idea where they are coming in from but they are everywhere. pooping all over the house

Agent: Ugh, that's really gross, John! 😖 I'm so sorry you're dealing with this. Since you're not sure about the entry points, we'd recommend having one of our expert technicians come over for an inspection. They can find those entry points for ya and help you get rid of those sneaky rodents. How does that sound? 🤔"""

In [3]:
print(conversation)

User: Hi name is John Smith and I am having an issue with rodents

Agent: Hey John, bummer about the rodent issue! I'm here to help you out. Can I ask you a few questions to get a better understanding of your situation? 😊

User: Sure that is fine

Agent: Awesome, thanks John! To start, can you tell me where you're located? It helps me figure out the best local solutions for your rodent problem. 😊

User: Yeah I am in Alexandria
Agent: Hey John, thanks for sharing! Alexandria is a lovely place - minus the rodents, of course. 😄 Now, could you let me know what kind of property you're dealing with, like a house or an apartment? That'll help me suggest the right approach.

User: Yes it is a single family home

Agent: Gotcha, John! Single-family homes can sometimes be a magnet for rodents. Don't worry, we've got your back. 😎 Have you noticed any specific areas inside or outside your home where these pesky rodents seem to be hanging out the most?

User: I am seeing them in my basement and my k

In [3]:
conversation = """
Lead: I am looking for a one time ant treatment.
Agent: Oh no! I'm sorry to hear that John. Luckily we can help with that :) Would you mind if I asked you a few questions so I can get a handle on your issue?
Lead: Sure, go ahead.
Agent: Great thanks! First how long has this been going on?
Lead: 2 Weeks
Agent: And how many ants are you seeing?
Lead: A lot! They are all over the kitchen.
Agent: OK don't worry I see this kind of thing all the time. How big is your house?
Lead: 2000 square feet
Agent: Got it. Give me just a second and let me run some numbers for you.
Agent: OK here's what I'm thinking:
"""

service = analyze_situation_sequential({ 
    "property_type": "single family",
    "square_footage": 1400,
    "acres": 1,
    "target": "ants",
    #"severity": "high", #solved
    "conversation_history": conversation
})

#eliminate targets by mapping to a list of targets id -> name (more about string parsing) - use a transform chain



> Entering new SequentialChain chain...
STEP: Getting situational experiences
Situational experiences found:  ['Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Ants will frequently come back and customers will benefit from ongoing services. One time services are not advisable but can be offered only if the customer doesn’t want a plan.', 'Ants will frequently come back and customers will benefit from ongoing services. One time services 

In [5]:
print(service['pitch'])

Thanks for sharing your ant issue! As I mentioned earlier, there may be some complexity to the situation, but we definitely have a great one-time service that can help. However, based on our experiences, ongoing services are recommend for ant issues just like yours. That's why I'd recommend our Castle Program. It's a bimonthly pest control program that covers pests and proactive rodent issues. With this program, you don't have to worry about ants, spiders, mild or moderate rodent issues, millipedes, mild roaches issues, crickets, centipedes, and other pests that may come your way because it forms a protective barrier around your home, adding extra prevention for termites and pests. Plus, you'll get a proactive annual termite inspection that reduces the risk of having termites for years undetected. It's the best pest control solution for issues like ants, spiders, millipedes, centipedes, and more. If you decide to get this ongoing plan, you'll only pay $149 for the initial treatment and

In [39]:
import datetime
import random

#a class for mocking a connection to pestpac database for testing purposes of the LLM
class PestPacConnection:
    def __init__(self, api_key, username, password):
        self.api_key = api_key
        self.username = username
        self.password = password
    
    #make sure the default start date is tomorrow's date and end date is tomorrows + 10
    def best_fit(self, lat, long, service, s_date=None, e_date=None, dow=None, time_rg=None):
        if s_date == None:
            s_date = datetime.date.today() + datetime.timedelta(days=1)
        if e_date == None:
            e_date = datetime.date.today() + datetime.timedelta(days=11)
        
        #generate 3 random dates between the start and end date and choose a random tech
        appointments = []
        techs = ['John Smith', 'Jayden Nicholas', 'Gregory Stepheson']
        time_ranges = ['9:00 AM - 12:00 PM', '12:00 PM - 3:00 PM', '3:00 PM - 6:00 PM']
        for i in range(3):
            #get 3 random dates between the start and end date
            date = random.randint(s_date.toordinal(), e_date.toordinal())
            date = datetime.date.fromordinal(date)
            tech = random.choice(techs)
            time = random.choice(time_ranges)
            appointments.append({'date': date, 'tech': tech, 'time': time})

        return appointments
    
    def create_service(self, loc_id, serv, bill_freq, bill_inf, recur_price, recur_time, tech, in_date, instr, in_time, initial_price):
        print('Creating new service')
        print(f' loc_id: {loc_id},\n serv: {serv},\n bill_freq: {bill_freq},\n bill_inf: {bill_inf},\n recur_price: {recur_price},\n recur_time: {recur_time},\n tech: {tech},\n in_date: {in_date},\n instr: {instr},\n in_time: {in_time},\n initial_price: {initial_price}')
        return True

    def create_loc(self, name, address, city, state, zip, phone, email):
        print('Creating new location')
        print(f' name: {name},\n address: {address},\n city: {city},\n state: {state},\n zip: {zip},\n phone: {phone},\n email: {email},\n')
        return True

pp = PestPacConnection(api_key='123', username='123', password='123')
pp.create_loc('George Schulz', '123 Elm Way', 'Gaithersburg', 'MD', '20878', '571-431-9531', 'georgeschulz33@gmail.com')

Creating new location
 name: George Schulz,
 address: 123 Elm Way,
 city: Gaithersburg,
 state: MD,
 zip: 20878,
 phone: 571-431-9531,
 email: georgeschulz33@gmail.com,



True

In [13]:
class Payments:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def get_product(self, program_name):
        programs = {
            'All in One': 'prod_MLignVDWxCwv98',
            'lawnshield': 'prod_NX8oqOKLA2tPIn',
            'castle': 'prod_NX8pSb35sDjLqE'        
        }

        return programs[program_name]
    
    def create_payment_link(self, service_name, initial):
        stripe.api_key = self.api_key
        service_id = self.get_product(service_name)
        unit_amount = int(initial * 100)
        #create the price for the service in the stripe dashboard
        price = stripe.Price.create(currency="usd", unit_amount=unit_amount, product=service_id)
        price_id = price.id
        #create the payment link
        link = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price': price_id,
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success',
            cancel_url='https://example.com/cancel',
        )
        return link.url


pay = Payments(api_key=os.getenv('STRIPE_KEY'))

In [15]:
pay.create_payment_link('castle', 84.99)

'https://checkout.stripe.com/c/pay/cs_test_a12BomIMePybf7XcJHkYeLnvvfFplRLN7i7cE5UtLZuGG7qJT27APCm9D1#fidkdWxOYHwnPyd1blpxYHZxWjA0TTJoRHZOaGtxTUlPdF9PR1VGYGdOXHMwVU4zNFVkXERqbWlITX82VHdmaVxtXFZSYHdOaGZTa0RUa1Y0ZEpjVF9CVV9KYzBnTEB9a1cxUmc0TGNwZEwyNTU2UTVmc1JgZicpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl'

In [ ]:
class Knowledge:
    def __init__(self):
        pass

    def find_best_service(self, pest, location):
        #find the best service for the pest and location
        return 'All in One'
    
    def assess_situation(self, pest, location):
        #find the best service for the pest and location
        return 'All in One'
    
    def get_price(self, service):
        #get the price for the service
        return 84.99
    
    

In [16]:
show_chat_log(agent.history)

*Initialization Prompt*: Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
------------------------------------
*User*: Hi name is John Smith and I am interested in pest control
------------------------------------
*Agent*: Hey John, nice to meet you! I'd love to help you out with any pest control issues you might be facing. Is it cool if I ask you a few questions to get a better understanding of your situation?
------------------------------------
*User*: Sure
------------------------------------
*Agent*: Awesome, thanks John! To start, what's your phone number and email address? This will help me get in touch with you if we need to chat more about your pest control needs.
------------------------------------
*User*: My phone is 571-431-9531 and email is georgeschulz33@gmail.com
------------------------------------
*Agent*: Thanks, George! Appreciate you sharing that info. Now, can you tell me your street address, city, state, and zip code? That way, we 

In [18]:
print(agent.issues.natural_language())

location: Master bedroom, front porch
special_instructions: Spiders are in multiple locations including the master bedroom and front porch. House is 3400 square feet.
pest_id: 2
square_footage: 3400

